## Import package

In [ ]:
import sys
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb
import time
import os

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

## Helper Function

In [ ]:
def readGroundtruths(path):
    """
    this function is used to read annotations from the txt file
    input: file path
    output: annotations
    """
    
    annotations = []
    
    with open(path) as file:
           for line in file:
                row = line.strip().split(";")
                annotations.append(row)
                
    return annotations

In [ ]:
def hsv_segmentation(img):
    """
    This function is used to perform HSV segmentation
    input: image
    output: mask of that image
    """
    
    # blur the image and convert the color range to HSV
    kernel_size = 5
    img_blur = cv.GaussianBlur(img, (kernel_size, kernel_size), 0)
    hsv_img = cv.cvtColor(img_blur, cv.COLOR_BGR2HSV)
    
    # get mask
    mask1 = cv.inRange(hsv_img, low_red, high_red)
    mask2 = cv.inRange(hsv_img, low_red2, high_red2)
    mask3 = cv.inRange(hsv_img, low_yellow, high_yellow)
    mask4 = cv.inRange(hsv_img, low_blue, high_blue)
    
    # combine all the masks 
    combined_mask = cv.add(cv.add(cv.add(mask1, mask2), mask3), mask4)
    
    # opening morphology
    kernel = np.ones((3,3), np.uint8)
    combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, kernel, iterations=3)

    return combined_mask

In [ ]:
def findContour(mask):
    """
    This function is used to find the largest contour
    input: mask in gray scale
    output: contour
    """
    
    mask_bgr = cv.cvtColor(mask, cv.COLOR_GRAY2BGR) 
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    #if no contour
    if len(contours) == 0:
        return None
    
    #find largest contour
    length = []
    for i in contours:
        length.append(len(i))
    max = np.argmax(length)
    contour = contours[max]
    
    return contour

In [ ]:
def getGroundtruth(imgName, contour, annotations, resized_WH):
    """
    this function is used to get predicted bounding box for original image size, actual bounding box and annotation for the image
    input: image name, selected contour, annotations and width and height of the resized image
    output: predicted and actual bounding box and the annotation of that image
    """
    
    #convert the bounding box of resized image to original image
    for annotation in annotations:
        if annotation[0] == imgName:
            predict_XYWH = getOriSizeBBox(contour, annotation, resized_WH) 
            break

    #get actual bounding box
    (x,y,x2,y2) = (int(annotation[3]), int(annotation[4]), int(annotation[5]), int(annotation[6]))
    w = x2 - x
    h = y2 - y
    truth_XYWH = (x,y,w,h)
        
    return predict_XYWH, truth_XYWH, annotation

In [ ]:
def getOriSizeBBox(contour, annotation, resized_WH):
    """
    this function is used to convert bounding box of the resized image to the original image
    input: contour, groundtruth, resized size
    output: bounding box for original size image
    """
    
    #information of the original image
    ori_width, ori_height = float(annotation[1]), float(annotation[2])
    
    #information of the resized image
    x, y, w, h = cv.boundingRect(contour) #bounding box for resized image
    resized_w, resized_h = resized_WH
    
    #calculate bounding box for original size image
    width_ratio = float(ori_width/resized_w)
    height_ratio = float(ori_height/resized_h)
    x *= width_ratio
    y *= height_ratio
    w *= width_ratio
    h *= height_ratio
        
    return (x,y,w,h)

In [ ]:
def accuracy(predictXYWH, truth_XYWH, annotation):
    """
    This function is used to calculate accuracy (Pixel accuracy, IoU), precision and recall
    input: predicted and actual starting point, width and height, annotation
    output: accuracy (pixel accuracy, IoU), precision and recall
    """
    
    # Exctract the value of 2 point of coordinates, width and height
    predicted_x, predicted_y, predicted_w, predicted_h = predictXYWH
    predicted_x2 = predicted_x + predicted_w
    predicted_y2 = predicted_y + predicted_h
    
    truth_x, truth_y, truth_w, truth_h = truth_XYWH
    truth_x2 = truth_x + truth_w
    truth_y2 = truth_y + truth_h
    
    ori_width, ori_height = float(annotation[1]), float(annotation[2])
    
    # Find the point of overlap area
    (x1, y1) = (max(predicted_x, truth_x), max(predicted_y, truth_y))  
    (x2, y2) = (min(predicted_x2, truth_x2), min(predicted_y2, truth_y2)) 
    
    # Calculate the area(overlapped, full image, predicted, actual and union)
    overlapped_w = x2 - x1
    overlapped_h = y2 - y1
    if overlapped_w > 0 and overlapped_h > 0:
        area_overlapped = overlapped_w * overlapped_h
    else:
        area_overlapped = 0
    
    area_img = ori_width * ori_height
    area_pred = predicted_w * predicted_h
    area_truth = truth_w * truth_h
    area_union = area_pred + area_truth - area_overlapped
    
    # True negative, false positive and false negative
    area_TN = area_img - area_union
    area_FP = area_union - area_truth
    area_FN = area_union - area_pred
    
    # Calculate the pixel accuracy, IoU, precision and recall
    pixel_acc = (area_overlapped + area_TN) / (area_overlapped + area_TN + area_FP + area_FN)
    IoU = area_overlapped / area_union
    
    if(area_overlapped != 0):
        precision = area_overlapped / (area_overlapped + area_FP)
        recall = area_overlapped / (area_overlapped + area_FN)
    else:
        precision, recall = 0, 0
    
    return IoU, pixel_acc, precision, recall

In [ ]:
def drawBbox(img, xywh, color, thickness):
    """
    This function is used to draw the bounding box (for showcasing)
    input: img in BGR
    output: img with bounding box
    """
    
    #if no contour
    if xywh is None:
        return img
        
    x, y, w, h = xywh
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    boundingBox = cv.rectangle(img, (x,y), (x+w, y+h), color, thickness)
    return boundingBox

## Run and segment all the images in the folder

In [ ]:
# HSV color range
low_red = np.array([0,40,30])
high_red = np.array([5, 245, 255])
low_red2 = np.array([119,40,30])
high_red2 = np.array([179, 245, 255])
low_yellow = np.array([10,117,100])
high_yellow = np.array([40, 255, 255])
low_blue = np.array([89,71,71])
high_blue = np.array([125, 240, 190])

# Locate the image folder path and the annotation file path
current_dir = os.getcwd()
img_path = os.path.join(current_dir, "70 test images files")
path = os.path.join(current_dir, "./TsignRecgTrain4170Annotation.txt")
annotations = readGroundtruths(path)

# set variable to 0 for counting total and timestamp the starting time
total_acc, total_IoU, total_pre, total_re, count, passed_count, print_count = 0, 0, 0, 0, 0, 0, 0
first_hstack = False
start_time = time.time()
interval = start_time
output, vertical = [], []

# loop through the images inside the folder
for filename in os.listdir(img_path):
        img = cv.imread(os.path.join(img_path,filename))
        if img is not None:
            # Resize the image
            img_resized = cv.resize(img, (200,200), interpolation = cv.INTER_CUBIC)
            img_ori = img_resized.copy()
            w, h, _ = img_resized.shape
            WH = (w, h)
            
            # Perform HSV color segmentation then Canny edge afterward
            res = hsv_segmentation(img_resized)
            edges = cv.Canny(res, 180, 255)
            contour = findContour(edges)
            
            # calculate the accuracy
            pred_XYWH, truth_XYWH, annotation = getGroundtruth(filename, contour, annotations, WH)
            IoU, acc, precision, recall = accuracy(pred_XYWH, truth_XYWH, annotation)
            total_acc += acc
            total_IoU += IoU
            total_pre += precision
            total_re += recall
            count += 1
            print_count += 1
            if(IoU > 0.8):
                passed_count += 1
                
            # Draw the bounding box to show at the end of the code
            x, y, w, h = cv.boundingRect(contour)
            draw_XYWH = (x, y, w, h)
            drawBbox(img_resized, draw_XYWH, (0,0,255), thickness=2)
            
            # Append the images into one
            append_image = np.hstack((img_ori, img_resized))
            if (print_count == 1):
                vertical = append_image
            else:
                vertical = np.vstack((vertical, append_image))
                if (print_count == 10):
                    if (first_hstack == False):
                        output = vertical
                        first_hstack = True
                    else:
                        output = np.hstack((output, vertical))
                    print_count = 0;
            
            # print the accuracy and computing time
            print(count, "\tAccuracy(Pixel, IoU):", format((acc*100), ".2f"), "%, ", format((IoU*100), ".2f"), "%", "\tComputing time:", time.time() - interval)
            interval = time.time()         

# calculate the average    
avg_acc = (total_acc / count)*100
avg_IoU = (total_IoU / count)*100
avg_pre = (total_pre / count)*100
avg_re = (total_re / count)*100
f_score = 2 * avg_pre* avg_re / (avg_pre + avg_re)

# Print out the result
print("Average (Pixel, IoU, precision, recall):", format(avg_acc, ".2f"), "%, ", format(avg_IoU, ".2f"), "%, ",  
      format(avg_pre, ".2f"), "%, ", format(avg_re, ".2f"), "%", "  F-score:",format(f_score, ".2f"), "%", 
      "\nNumber of images passed 80% IoU:", passed_count, "  Total:", count, "  total computing time:", format(interval - start_time, ".4f"), "seconds")

# write all the segmented result in one image
cv.imwrite(os.path.join(current_dir, 'output.png'), output)

## Test the code

In [ ]:
# HSV color range
low_red = np.array([0,40,30])
high_red = np.array([5, 245, 255])
low_red2 = np.array([119,40,30])
high_red2 = np.array([179, 245, 255])
low_yellow = np.array([10,117,100])
high_yellow = np.array([40, 255, 255])
low_blue = np.array([89,71,71])
high_blue = np.array([125, 240, 190])

# Locate the annotation file path and the image folder path
current_dir = os.getcwd()
path = os.path.join(current_dir, "./TsignRecgTrain4170Annotation.txt")
annotations = readGroundtruths(path)

img_path = os.path.join(current_dir, "70 test images files")
imgName = "006_0017.png"
img = cv.imread(os.path.join(img_path,imgName))
img_ori = img.copy()

# Resize the image
img_resized = cv.resize(img, (200,200), interpolation = cv.INTER_CUBIC)
w, h, _ = img_resized.shape
WH = (w, h)

# Perform HSV color segmentation then Canny edge afterward
res = hsv_segmentation(img_resized)
edges = cv.Canny(res, 180, 255)
contour = findContour(edges)

# calculate the accuracy
pred_XYWH, truth_XYWH, annotation = getGroundtruth(imgName, contour, annotations, WH)
IoU, acc, precision, recall = accuracy(pred_XYWH, truth_XYWH, annotation)

print("accuracy (Pixel, IoU)", format(acc*100, ".2f"), "%, ", format(IoU*100, ".2f"), "% ", " precision:", 
      format(precision*100, ".2f"), "% ", " recall:", format(recall*100, ".2f"), "% ",)

drawBbox(img_ori, pred_XYWH, (0,0,255), thickness=2) #red = predicted
drawBbox(img_ori, truth_XYWH, (0,255,0), thickness=2) #green = groundtruth
append_image = np.hstack((img, img_ori))

cv.imshow('img', append_image)
cv.waitKey(0)
cv.destroyAllWindows()